In [1]:
import os
from time import sleep

import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
AV1 = os.getenv("AV1")
AV2 = os.getenv("AV2")
AV3 = os.getenv("AV3")
AV4 = os.getenv("AV4")

df = pd.read_csv("./predictions.csv")
tickers = df["ticker"].unique()

In [10]:
def get_bitcoin_data():
    """
    Fetch historical Bitcoin price from multiple sources.
    Tries in order: blockchain.info, CoinGecko, Alpha Vantage
    """
    # Try blockchain.info first (free, no API key required)
    print("Trying blockchain.info for Bitcoin price...")
    try:
        url = "https://api.blockchain.info/charts/market-price"
        params = {"timespan": "all", "format": "json", "sampled": "true"}
        headers = {
            "accept": "application/json",
            "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
        }
        r = requests.get(url, params=params, headers=headers, timeout=30)
        r.raise_for_status()
        data = r.json()

        if (
            isinstance(data, dict)
            and "values" in data
            and isinstance(data["values"], list)
        ):
            btc_dict = {}
            for item in data["values"]:
                if isinstance(item, dict):
                    x = item.get("x")
                    y = item.get("y")
                    if x is not None and y is not None:
                        try:
                            date = pd.to_datetime(int(x), unit="s").to_period("Q")
                            quarter = date.strftime("%Yq%q")
                            btc_dict[quarter] = float(y)
                        except (ValueError, TypeError, OverflowError):
                            continue
            if btc_dict:
                print(
                    f"Successfully fetched {len(btc_dict)} quarters from blockchain.info"
                )
                return btc_dict
    except Exception as e:
        print(f"blockchain.info BTC price error: {e}")

    # Try CoinGecko (may require API key now)
    print("Trying CoinGecko for Bitcoin price...")
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {"vs_currency": "usd", "days": "max", "interval": "daily"}
    headers = {
        "accept": "application/json",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
    }

    try:
        r = requests.get(url, params=params, headers=headers, timeout=30)
        r.raise_for_status()
        data = r.json()

        if "prices" in data and isinstance(data["prices"], list):
            btc_dict = {}
            for item in data["prices"]:
                if isinstance(item, list) and len(item) >= 2:
                    timestamp, price = item[0], item[1]
                    if price is not None and isinstance(price, (int, float)):
                        date = pd.to_datetime(timestamp, unit="ms").to_period("Q")
                        quarter = date.strftime("%Yq%q")
                        btc_dict[quarter] = float(price)
            if btc_dict:
                print(f"Successfully fetched {len(btc_dict)} quarters from CoinGecko")
                return btc_dict

    except requests.exceptions.RequestException as e:
        print(f"CoinGecko API error: {e}")
    except (ValueError, KeyError, TypeError) as e:
        print(f"CoinGecko parsing error: {e}")

    # Fallback: Try Alpha Vantage for BTC price
    print("Trying Alpha Vantage for Bitcoin data...")
    try:
        av_key = os.getenv("AV1")
        if av_key:
            url = "https://www.alphavantage.co/query"
            params = {
                "function": "DIGITAL_CURRENCY_DAILY",
                "symbol": "BTC",
                "market": "USD",
                "apikey": av_key,
            }
            headers = {
                "accept": "application/json",
                "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
            }
            r = requests.get(url, params=params, headers=headers, timeout=30)
            r.raise_for_status()
            data = r.json()

            # Debug: print response keys
            print(f"Alpha Vantage BTC response keys: {list(data.keys())}")

            time_series_key = "Time Series (Digital Currency Daily)"
            if time_series_key in data:
                btc_dict = {}
                for date_str, values in data[time_series_key].items():
                    try:
                        # Use "4a. close (USD)" for the close price
                        price = float(values.get("4a. close (USD)", 0))
                        if price > 0:
                            date = pd.to_datetime(date_str).to_period("Q")
                            quarter = date.strftime("%Yq%q")
                            btc_dict[quarter] = price
                    except (ValueError, TypeError):
                        continue
                if btc_dict:
                    print(
                        f"Successfully fetched {len(btc_dict)} quarters from Alpha Vantage"
                    )
                    return btc_dict
    except Exception as e:
        print(f"Alpha Vantage BTC error: {e}")

    print("Warning: Could not fetch Bitcoin price data from any source")
    return {}


def get_hashrate_data():
    """
    Fetch Bitcoin network hashrate from blockchain.info.
    Response format: {"status": "ok", "name": "...", "unit": "...", "values": [...]}
    Each value: {"x": unix_timestamp, "y": hashrate_value}
    """
    url = "https://api.blockchain.info/charts/hash-rate"
    params = {"timespan": "all", "format": "json", "sampled": "true"}
    headers = {
        "accept": "application/json",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
    }

    try:
        r = requests.get(url, params=params, headers=headers, timeout=30)
        r.raise_for_status()
        data = r.json()

        # Validate response structure
        if not isinstance(data, dict):
            print(f"Unexpected hashrate response type: {type(data)}")
            return {}

        if "values" not in data:
            print(f"Hashrate response missing 'values'. Keys: {list(data.keys())}")
            return {}

        values = data["values"]
        if not isinstance(values, list):
            print(f"Hashrate 'values' is not a list: {type(values)}")
            return {}

        hashrate_dict = {}
        for item in values:
            if not isinstance(item, dict):
                continue

            x = item.get("x")
            y = item.get("y")

            if x is None or y is None:
                continue

            try:
                date = pd.to_datetime(int(x), unit="s").to_period("Q")
                quarter = date.strftime("%Yq%q")
                hashrate_dict[quarter] = float(y)
            except (ValueError, TypeError, OverflowError) as e:
                continue

        return hashrate_dict

    except requests.exceptions.Timeout:
        print("Timeout fetching hashrate data")
        return {}
    except requests.exceptions.RequestException as e:
        print(f"Request error fetching hashrate: {e}")
        return {}
    except ValueError as e:
        print(f"JSON decode error for hashrate: {e}")
        return {}
    except Exception as e:
        print(f"Unexpected error fetching hashrate: {e}")
        return {}

In [15]:
# Pre-fetch crypto control data (time-based, same for all tickers)
print("Fetching Bitcoin price data...")
btc_prices = get_bitcoin_data()
sleep(2)

print("Fetching hashrate data...")
hashrate_data = get_hashrate_data()
sleep(2)

print(f"Bitcoin data quarters: {len(btc_prices)}")
print(f"Hashrate data quarters: {len(hashrate_data)}")

Fetching Bitcoin price data...
Trying blockchain.info for Bitcoin price...
Successfully fetched 68 quarters from blockchain.info
Fetching hashrate data...
Bitcoin data quarters: 68
Hashrate data quarters: 68


In [ ]:
def get_fiscal_quarter(date_str):
    """Convert date string to fiscal quarter format (YYYYqQ)."""
    try:
        date = pd.to_datetime(date_str).to_period("Q")
        return date.strftime("%Yq%q")
    except Exception:
        return None


def safe_float(value, default=None):
    """
    Safely convert value to float, handling various edge cases.
    Alpha Vantage returns "None" as string for missing data.
    """
    if value is None:
        return default
    if isinstance(value, (int, float)):
        return float(value)
    if isinstance(value, str):
        value = value.strip()
        if value in ("None", "", "-", "N/A", "null", "nan"):
            return default
        try:
            return float(value)
        except ValueError:
            return default
    return default


def validate_alpha_vantage_response(data, ticker, endpoint_name):
    """
    Validate Alpha Vantage API response and return (is_valid, error_message).

    Alpha Vantage error responses:
    - {"Information": "..."} - General info/premium required
    - {"Error Message": "..."} - Invalid API call
    - {"Note": "..."} - Rate limit exceeded (5 calls/min free tier)
    """
    if not isinstance(data, dict):
        return False, f"Response is not a dict: {type(data)}"

    if "Information" in data:
        return False, f"API Info: {data['Information'][:100]}"

    if "Error Message" in data:
        return False, f"API Error: {data['Error Message'][:100]}"

    if "Note" in data:
        return False, f"Rate Limit: {data['Note'][:100]}"

    return True, None


def get_quarterly_roa(ticker, api_key, btc_prices, hashrate_data):
    """
    Fetch quarterly ROA and control variables for a given ticker.

    Alpha Vantage API endpoints used:
    1. INCOME_STATEMENT - Returns {symbol, annualReports[], quarterlyReports[]}
    2. BALANCE_SHEET - Returns {symbol, annualReports[], quarterlyReports[]}
    3. TIME_SERIES_MONTHLY_ADJUSTED - Returns {"Meta Data": {...}, "Monthly Adjusted Time Series": {...}}

    quarterlyReports fields (partial):
    - fiscalDateEnding: "YYYY-MM-DD"
    - netIncome: "123456789" or "None"
    - totalAssets: "123456789" or "None"
    - shortLongTermDebtTotal / totalDebt: debt values
    - cashAndCashEquivalentsAtCarryingValue: cash
    - totalShareholderEquity: equity
    - commonStockSharesOutstanding: shares
    """
    base_url = "https://www.alphavantage.co/query"

    headers = {
        "accept": "application/json",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    }

    # 1. Get Income Statement Data
    params_income = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key,
    }

    try:
        r_income = requests.get(
            base_url, params=params_income, headers=headers, timeout=30
        )
        r_income.raise_for_status()
        data_income = r_income.json()
    except requests.exceptions.Timeout:
        print(f"Timeout fetching {ticker} Income Statement")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request error fetching {ticker} Income Statement: {e}")
        return None
    except ValueError as e:
        print(f"JSON decode error for {ticker} Income Statement: {e}")
        return None

    is_valid, error_msg = validate_alpha_vantage_response(
        data_income, ticker, "Income Statement"
    )
    if not is_valid:
        print(f"{ticker} Income Statement: {error_msg}")
        return None

    if "quarterlyReports" not in data_income:
        print(
            f"{ticker}: No quarterlyReports in Income Statement. Keys: {list(data_income.keys())}"
        )
        return None

    if not isinstance(data_income["quarterlyReports"], list):
        print(f"{ticker}: quarterlyReports is not a list")
        return None

    sleep(5)

    # 2. Get Balance Sheet Data
    params_balance = {"function": "BALANCE_SHEET", "symbol": ticker, "apikey": api_key}

    try:
        r_balance = requests.get(
            base_url, params=params_balance, headers=headers, timeout=30
        )
        r_balance.raise_for_status()
        data_balance = r_balance.json()
    except requests.exceptions.Timeout:
        print(f"Timeout fetching {ticker} Balance Sheet")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request error fetching {ticker} Balance Sheet: {e}")
        return None
    except ValueError as e:
        print(f"JSON decode error for {ticker} Balance Sheet: {e}")
        return None

    is_valid, error_msg = validate_alpha_vantage_response(
        data_balance, ticker, "Balance Sheet"
    )
    if not is_valid:
        print(f"{ticker} Balance Sheet: {error_msg}")
        return None

    if "quarterlyReports" not in data_balance:
        print(
            f"{ticker}: No quarterlyReports in Balance Sheet. Keys: {list(data_balance.keys())}"
        )
        return None

    sleep(5)

    # 3. Get Monthly Stock Price Data
    params_price = {
        "function": "TIME_SERIES_MONTHLY_ADJUSTED",
        "symbol": ticker,
        "apikey": api_key,
    }

    price_series = {}
    try:
        r_price = requests.get(
            base_url, params=params_price, headers=headers, timeout=30
        )
        r_price.raise_for_status()
        data_price = r_price.json()

        is_valid, error_msg = validate_alpha_vantage_response(
            data_price, ticker, "Stock Price"
        )
        if is_valid:
            price_series = data_price.get("Monthly Adjusted Time Series", {})
        else:
            print(f"Warning: {ticker} Stock Price: {error_msg}")
    except Exception as e:
        print(f"Warning: Could not fetch {ticker} Stock Price: {e}")

    if not price_series:
        print(f"Warning: No price data for {ticker}, P/B ratio will be unavailable")

    # 4. Process and Merge Data
    income_dict = {}
    for item in data_income.get("quarterlyReports", []):
        if not isinstance(item, dict):
            continue
        fiscal_date = item.get("fiscalDateEnding")
        if fiscal_date:
            quarter = get_fiscal_quarter(fiscal_date)
            if quarter:
                income_dict[quarter] = item

    balance_dict = {}
    for item in data_balance.get("quarterlyReports", []):
        if not isinstance(item, dict):
            continue
        fiscal_date = item.get("fiscalDateEnding")
        if fiscal_date:
            quarter = get_fiscal_quarter(fiscal_date)
            if quarter:
                balance_dict[quarter] = item

    roa_data = []

    for quarter, income_report in income_dict.items():
        if quarter not in balance_dict:
            continue

        balance_report = balance_dict[quarter]

        try:
            # Core financials
            net_income = safe_float(income_report.get("netIncome"), 0)
            total_assets = safe_float(balance_report.get("totalAssets"), 0)

            # Skip if no total assets (can't calculate ROA)
            if total_assets is None or total_assets <= 0:
                print(f"Skipping {ticker} {quarter}: total_assets={total_assets}")
                continue

            # Firm Controls - raw values
            # Try multiple field names for debt (Alpha Vantage varies by company)
            total_debt = safe_float(
                balance_report.get("shortLongTermDebtTotal")
                or balance_report.get("longTermDebt")
                or balance_report.get("totalDebt"),
                0,
            )

            # Cash field
            cash = safe_float(
                balance_report.get("cashAndCashEquivalentsAtCarryingValue")
                or balance_report.get("cashAndShortTermInvestments")
                or balance_report.get("cash"),
                0,
            )

            shareholder_equity = safe_float(
                balance_report.get("totalShareholderEquity"), 0
            )
            shares_outstanding = safe_float(
                balance_report.get("commonStockSharesOutstanding"), 0
            )

            # Get stock price for P/B calculation (monthly data for end of quarter)
            fiscal_date = income_report.get("fiscalDateEnding")
            stock_price = None
            if fiscal_date and price_series:
                month_str = pd.to_datetime(fiscal_date).strftime("%Y-%m")
                for date_key in price_series:
                    if date_key.startswith(month_str):
                        stock_price = safe_float(
                            price_series[date_key].get("5. adjusted close")
                        )
                        break

            # Calculate ROA
            roa = net_income / total_assets

            # Calculate firm controls
            log_assets = np.log(total_assets) if total_assets > 0 else None
            leverage = total_debt / total_assets if total_assets > 0 else None
            liquidity = cash / total_assets if total_assets > 0 else None

            # P/B Ratio = Market Cap / Book Value (Shareholder Equity)
            market_cap = None
            pb_ratio = None
            if (
                stock_price is not None
                and stock_price > 0
                and shares_outstanding is not None
                and shares_outstanding > 0
            ):
                market_cap = stock_price * shares_outstanding
                if shareholder_equity and shareholder_equity > 0:
                    pb_ratio = market_cap / shareholder_equity

            # Crypto controls (time-based)
            btc_price = btc_prices.get(quarter) if btc_prices else None
            log_btc_price = np.log(btc_price) if btc_price and btc_price > 0 else None

            hashrate = hashrate_data.get(quarter) if hashrate_data else None
            log_hashrate = np.log(hashrate) if hashrate and hashrate > 0 else None

            roa_data.append(
                {
                    "ticker": ticker,
                    "fiscal_quarter": quarter,
                    "fiscal_date": fiscal_date,
                    "net_income": net_income,
                    "total_assets": total_assets,
                    "roa": roa,
                    "roa_percent": round(roa * 100, 8),
                    # Firm Controls
                    "log_assets": log_assets,
                    "total_debt": total_debt,
                    "leverage": leverage,
                    "cash": cash,
                    "liquidity": liquidity,
                    "stock_price": stock_price,
                    "shares_outstanding": shares_outstanding,
                    "market_cap": market_cap,
                    "shareholder_equity": shareholder_equity,
                    "pb_ratio": pb_ratio,
                    # Crypto Controls (Time Controls)
                    "btc_price": btc_price,
                    "log_btc_price": log_btc_price,
                    "hashrate": hashrate,
                    "log_hashrate": log_hashrate,
                }
            )
        except ZeroDivisionError:
            print(f"Division error for {ticker} {quarter}")
            continue
        except Exception as e:
            print(f"Error processing {ticker} {quarter}: {type(e).__name__}: {e}")
            continue

    df = pd.DataFrame(roa_data)
    if not df.empty:
        df["fiscal_date"] = pd.to_datetime(df["fiscal_date"])
        df = df.sort_values(by="fiscal_date", ascending=False).reset_index(drop=True)

    return df

In [25]:
api_keys = [AV1, AV2, AV3, AV4]

# For testing, limit to first 3 tickers
# test_tickers = tickers[:3]
# print(f"Processing {len(test_tickers)} tickers for testing: {test_tickers}")

all_roa_dfs = []
for i, ticker in enumerate(tickers):
    key_index = (i // 10) % len(api_keys)
    api_key = api_keys[key_index]
    print(f"Processing {ticker}...")
    roa_df = get_quarterly_roa(ticker, api_key, btc_prices, hashrate_data)
    sleep(5)
    if roa_df is not None:
        all_roa_dfs.append(roa_df)

combined_df = (
    pd.concat(all_roa_dfs, ignore_index=True) if all_roa_dfs else pd.DataFrame()
)

Processing ABTS...
Skipping ABTS 2019q4: total_assets=0
Skipping ABTS 2013q4: total_assets=0
Processing ARLP...
Processing ABTC...
ABTC: No quarterlyReports in Income Statement. Keys: []
Processing APLD...
Skipping APLD 2021q1: total_assets=0
Skipping APLD 2020q4: total_assets=0
Skipping APLD 2020q3: total_assets=0
Skipping APLD 2002q3: total_assets=0
Processing ARBK...
Processing BTOG...
Skipping BTOG 2016q4: total_assets=0
Skipping BTOG 2016q3: total_assets=0
Processing BTDR...
Skipping BTDR 2022q3: total_assets=0
Skipping BTDR 2020q2: total_assets=0
Skipping BTDR 2020q1: total_assets=0
Processing BITF...
Processing FUFU...
Skipping FUFU 2021q2: total_assets=0
Skipping FUFU 2021q1: total_assets=0
Processing BMNR...
Skipping BMNR 2020q4: total_assets=0
Processing CAN...
CAN Income Statement: Rate Limit: We have detected your API key as J8MDZQ3H44NVM720 and our standard API rate limit is 25 requests per
Processing CANG...
Skipping CANG 2017q2: total_assets=0
Skipping CANG 2017q1: total

/tmp/ipykernel_47774/911521243.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(all_roa_dfs, ignore_index=True) if all_roa_dfs else pd.DataFrame()


In [26]:
combined_df.to_csv("./roa11.csv", index=False)

In [27]:
combined_df.head().to_csv("./roa11-head.csv", index=False)